In [0]:
# %sql
# DROP TABLE dimaccount;
# DROP TABLE dimpolicy;
# DROP TABLE dimaccountingperiod;
# DROP TABLE dimcurrency;
# DROP TABLE dimentity;
# DROP TABLE dimlob;
# DROP TABLE dimyoa;
# DROP TABLE facteurobase;
# DROP TABLE watermark;

#### Setting the Credentials

In [0]:
%run /Workspace/ProjectMedallion/nb_setup_cred

#### Importing Libs

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import year, to_date, substring, col, lit, month, year, when, max
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row
import calendar
from datetime import datetime
from delta.tables import DeltaTable

#### Reading the Source Data from Silver

In [0]:
df = spark.read.format("delta")\
               .load(ds_path)

#### Creating tables if not exists

In [0]:
%run /Workspace/ProjectMedallion/nb_tablecreation

### Creating Function for each Dimension and Facts Tables

#### dimPolicy

In [0]:
def loadDimPolicy():

  dimPolicy = spark.table("dimPolicy").select("PolicyReference", "InceptionDate")

  new_records_df = df.join(dimPolicy, on=["PolicyReference", "InceptionDate"], how="left_anti").select("PolicyReference", "InceptionDate")

  dimPolicy = dimPolicy.unionByName(new_records_df)

  dimPolicy.write.mode('append').saveAsTable("dimPolicy")
  


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
loadDimPolicy()

#### dimEntity

In [0]:
def loadDimEntity():

  dimEntity = spark.table("dimEntity").select("SyndicateNumber")

  # Step 2: Find new SyndicateNumbers that are not in dimEntity
  new_records_df = (
      df.select("SyndicateNumber")
      .distinct()  # Ensure only unique SyndicateNumbers from df
      .join(dimEntity, on=["SyndicateNumber"], how="left_anti")  # Exclude existing records
  )

  # Step 3: Append new SyndicateNumbers to dimEntity
  if not new_records_df.isEmpty():  # Check if there are new records
      dimEntity = dimEntity.unionByName(new_records_df)  # Union the new records
      dimEntity.write.mode("overwrite").saveAsTable("dimEntity")  # Save the updated table
  else:
      print("No new SyndicateNumbers to append.")

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
loadDimEntity()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

#### dimCurrency

In [0]:
def loadDimCurrency():
    
    dimCurrency = spark.table("dimCurrency").select("PolicySettlementCurrency")
    
    # Find new PolicySettlementCurrency values that are not in dimCurrency
    new_records_df = (
        df.select("PolicySettlementCurrency")  
        .distinct()  
        .join(dimCurrency, on=["PolicySettlementCurrency"], how="left_anti")  # Exclude existing records
    )
    
    # Append new PolicySettlementCurrency values to dimCurrency
    if not new_records_df.isEmpty():  
        dimCurrency = dimCurrency.unionByName(new_records_df)  
        dimCurrency.write.mode("overwrite").saveAsTable("dimCurrency")
    else:
        print("No new PolicySettlementCurrency values to append.")


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
loadDimCurrency()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

#### dimYOA

In [0]:
def loadDimYOA():

  # Step 1: Read existing dimYOA table
  dimYOA = spark.table("dimYOA").select("PolicyYOA")

  # Step 2: Find new PolicyYOA values that are not in dimYOA
  new_records_df = (
      df.selectExpr("CAST(PolicyYOA AS BIGINT) AS PolicyYOA")  # Ensure correct data type
      .distinct()  # Ensure unique PolicyYOA values from df
      .join(dimYOA, on=["PolicyYOA"], how="left_anti")  # Exclude existing records
  )

  # Step 3: Append new PolicyYOA values to dimYOA
  if not new_records_df.isEmpty():  # Check if there are new records
      dimYOA = dimYOA.unionByName(new_records_df)  # Union the new records
      dimYOA.write.mode("overwrite").saveAsTable("dimYOA")  # Save the updated table
  else:
      print("No new PolicyYOA values to append.")


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
loadDimYOA()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

#### dimLOB

In [0]:
def loadDimLOB():

  dimLOB = spark.table("dimLOB").select("TriFocusGroup", "Department")

  # Step 2: Find new TriFocusGroup and Department combinations that are not in dimLOB
  new_records_df = (
      df.select("TriFocusGroup", "Department")
      .distinct()  # Ensure only unique combinations from df
      .join(dimLOB, on=["TriFocusGroup", "Department"], how="left_anti")  # Exclude existing records
  )

  # Step 3: Append new combinations to dimLOB
  if not new_records_df.isEmpty():  # Check if there are new records
      dimLOB = dimLOB.unionByName(new_records_df)  # Union the new records
      dimLOB.write.mode("overwrite").saveAsTable("dimLOB")  # Save the updated table
  else:
      print("No new TriFocusGroup and Department combinations to append.")

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
loadDimLOB()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

#### dimAccount

In [0]:
def loadDimAccount():

    schema = StructType([
    StructField("AccountCode", StringType(), True),
    StructField("AccountType", StringType(), True)
    ])

    new_data = [['GWP-B', 'Binder'], ['GWP-NB', 'Non Binder']]

    df = spark.createDataFrame(new_data, schema=schema)

    df.write.format("delta").mode("append").saveAsTable("dimAccount")

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
dimaccount = spark.table("dimAccount")

if dimaccount.rdd.isEmpty():
    loadDimAccount()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

#### dimAccountingPeriod

In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, IntegerType, StringType
# import calendar

# def loadDimAccountingPeriod(start_month, start_year, duration_years):
#     """
#     Inserts data into the dimAccountingPeriod Delta table for a range of months and years.

#     :param start_month: Starting month (1-12)
#     :param start_year: Starting year (e.g., 2016)
#     :param duration_years: Duration in years (e.g., 6)
#     """
#     # Calculate the end year
#     end_year = start_year + duration_years - 1

#     # Generate data for the specified period
#     data = []
#     for year in range(start_year, end_year + 1):
#         for month in range(1, 13):
#             if year == start_year and month < start_month:
#                 continue  # Skip months before the start month in the first year
#             month_name = calendar.month_name[month]
#             data.append((month, month_name, str(year)))

#     # Define the schema
#     schema = StructType([
#         StructField("AccountingMonth", IntegerType(), True),
#         StructField("MonthName", StringType(), True),
#         StructField("YearName", StringType(), True)
#     ])

#     # Create a DataFrame
#     df = spark.createDataFrame(data, schema=schema)

#     # Write the DataFrame to the Delta table
#     df.write.format("delta").mode("append").saveAsTable("dimAccountingPeriod")

In [0]:
def loadDimAccountingPeriod():
    """
    Dynamically determines the range of years from the 'InceptionDate' column of the original DataFrame,
    generates missing data, and appends unique records to dimAccountingPeriod.
    """
    # Load the existing dimAccountingPeriod table or create it if it doesn't exist
    
    dimAccountingPeriod = spark.table("dimAccountingPeriod")
    

    # Extract min and max year from 'InceptionDate' in the source DataFrame
    min_year = df.select(
        year(to_date("InceptionDate", "dd-MM-yyyy HH:mm")).alias("Year")
    ).agg({"Year": "min"}).collect()[0][0]
    max_year = df.select(
        year(to_date("InceptionDate", "dd-MM-yyyy HH:mm")).alias("Year")
    ).agg({"Year": "max"}).collect()[0][0]

    # Generate data for the range of years
    data = []
    for yr in range(min_year, max_year + 1):  # Use 'yr' to avoid conflicts with 'year' function
        for month in range(1, 13):  # Always include all months from January to December
            month_name = calendar.month_name[month]
            data.append((month, month_name, str(yr)))

    # Define the schema
    schema = StructType([
        StructField("AccountingMonth", IntegerType(), True),
        StructField("MonthName", StringType(), True),
        StructField("YearName", StringType(), True)
    ])

    # Create a DataFrame with generated data
    new_df = spark.createDataFrame(data, schema=schema)

    # Remove duplicates by excluding records that already exist in the Delta table
    updated_df = new_df.join(dimAccountingPeriod, ["AccountingMonth", "MonthName", "YearName"], "left_anti")

    # Append unique data to the Delta table
    if not updated_df.rdd.isEmpty():
        updated_df.write.format("delta").mode("append").saveAsTable("dimAccountingPeriod")


In [0]:
loadDimAccountingPeriod()

#### watermark

In [0]:
def get_default_watermark_date():

    watermark = spark.table("watermark")

    if watermark.rdd.isEmpty():
        print("Watermark DataFrame is empty. Initializing with default date (01-01-1900).")
        default_date = "01-01-1900"
        new_watermark = spark.createDataFrame([Row(update_date=default_date)])
        new_watermark.write.mode("overwrite").saveAsTable("watermark")
    
    # Reload the watermark table after initialization
    return spark.table("watermark")

def update_watermark_date():

    watermark = spark.table("watermark")

    # df = df.withColumn("report_date", date_format(to_date("report_date", "yyyy-MM-dd"), "yyyy-MM-dd"))
    # watermark = watermark.withColumn("update_date", date_format(to_date("update_date", "yyyy-MM-dd"), "yyyy-MM-dd"))

    df_max_date = df.agg(max("report_date").alias("max_report_date")).collect()[0]["max_report_date"]
    watermark_max_date = watermark.agg(max("update_date").alias("max_update_date")).collect()[0]["max_update_date"]

    # print(f"Max date in df: {df_max_date}")
    # print(type(df_max_date))
    # print(f"Max date in watermark: {watermark_max_date}")
    # print(type(watermark_max_date))
    
    df_date = datetime.strptime(df_max_date, '%d-%m-%Y') 
    watermark_date = datetime.strptime(watermark_max_date, '%d-%m-%Y')

    if df_date and df_date > watermark_date:
        print(f"New max date found: {df_max_date}. Updating watermark table.")
        new_row = spark.createDataFrame([(df_max_date,)], ["update_date"])
        watermark = watermark.union(new_row)

        watermark.write.mode("overwrite").saveAsTable("watermark")

#### factEuroBase

##### using Spark SQL

In [0]:
# def loadEuroBase():

#   factEuroBase = spark.table("factEuroBase").select("MOP","GrossWrittenPremium")

#   new_records_df = df.join(factEuroBase, on=["MOP","GrossWrittenPremium"], how="left_anti").select("MOP","GrossWrittenPremium")

#   data = []
#   data.append(['fk_AccountID','fk_PolicyID','fk_EntityID','fk_LOBID','fk_YOAID','fk_CurrencyID','fk_PeriodID','MOP','GrossWrittenPremium'])

#   for row in new_records_df.collect():

#     fk_AccountID = spark.sql(f"""SELECT 
#                                 a.pk_AccountID
#                              FROM 
#                                 dimAccount a
#                             JOIN 
#                                 df 
#                             ON a.AccountType = CASE 
#                                                     WHEN df.Binder == 1 THEN 'Binder' ELSE 'Non Binder' 
#                                                 END 
#                              WHERE df.MOP = '{row['MOP']}' 
#                             AND df.GrossWrittenPremium = '{row['GrossWrittenPremium']}'""")


#     fk_PolicyID = spark.sql(f""" WITH Policy AS (
#                               SELECT 
#                                   PolicyReference,
#                                   InceptionDate
#                               FROM 
#                                   df
#                               WHERE 
#                                   MOP = '{row['MOP']}' AND 
#                                   GrossWrittenPremium = '{row['GrossWrittenPremium']}'
#                           )
#                           SELECT 
#                               p.pk_PolicyID
#                           FROM 
#                               dimPolicy p
#                           JOIN 
#                               Policy y
#                           ON 
#                               p.PolicyReference = y.PolicyReference AND
#                               p.InceptionDate = y.InceptionDate""")
    

#     fk_EntityID  = spark.sql(f"""SELECT pk_EntityID
#                                 FROM dimentity d
#                                 JOIN df ON d.SyndicateNumber = df.SyndicateNumber
#                                 WHERE df.MOP = '{row['MOP']}' AND df.GrossWrittenPremium = '{row['GrossWrittenPremium']}'""")
    

#     fk_LOBID  = spark.sql(f"""WITH cte AS (
#                             SELECT 
#                                 TrifocusGroup , 
#                                 Department
#                             FROM 
#                                 df
#                             WHERE 
#                                 MOP = '{row['MOP']}' AND 
#                                 GrossWrittenPremium = '{row['GrossWrittenPremium']}'
#                             )
#                             SELECT 
#                                 pk_LOBID
#                             FROM 
#                                 dimlob l
#                             JOIN 
#                                 cte c
#                             ON 
#                                 l.TrifocusGroup = c.TrifocusGroup AND 
#                                 l.Department = c.Department""")

#     fk_YOAID  = spark.sql(f"""SELECT 
#                                 pk_YOAID
#                             FROM 
#                                 dimyoa y
#                             JOIN 
#                                 df 
#                             ON 
#                                 df.PolicyYOA = y.PolicyYOA
#                             WHERE 
#                                 df.MOP = '{row['MOP']}' AND df.GrossWrittenPremium = '{row['GrossWrittenPremium']}'
#                             LIMIT 1
#                             """)
#     fk_CurrencyID = spark.sql(f"""SELECT 
#                                     pk_CurrencyID
#                                 FROM 
#                                     dimcurrency c
#                                 JOIN 
#                                     df 
#                                 ON 
#                                     df.PolicySettlementCurrency = c.PolicySettlementCurrency
#                                 WHERE 
#                                     df.MOP = '{row['MOP']}' AND df.GrossWrittenPremium = '{row['GrossWrittenPremium']}'""")
    
#     fk_PeriodID = spark.sql(f"""WITH cte AS (
#                                 SELECT  
#                                         DATE_FORMAT(TO_DATE(SUBSTRING(InceptionDate, 1, 10), 'dd-MM-yyyy'), 'M') AS Month,
#                                         DATE_FORMAT(TO_DATE(SUBSTRING(InceptionDate, 1, 10), 'dd-MM-yyyy'), 'y') AS Year
#                                 FROM   
#                                     df
#                                 WHERE 
#                                     MOP = '{row['MOP']}' AND 
#                                     GrossWrittenPremium = '{row['GrossWrittenPremium']}'
#                             )
#                             SELECT 
#                                 pk_PeriodID
#                             FROM 
#                                 dimaccountingperiod p 
#                             JOIN 
#                                 cte c
#                             ON p.YearName = c.Year AND p.AccountingMonth = c.Month""")

    
#     data.append([fk_AccountID,fk_PolicyID,fk_EntityID,fk_LOBID,fk_YOAID,fk_CurrencyID,fk_PeriodID,row['MOP'],row['GrossWrittenPremium']])

#     exit
#   factEuroBase = factEuroBase.unionByName(data)

#   factEuroBase.write.mode('append').saveAsTable("factEuroBase")

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

##### using Pyspark

In [0]:
# def loadEuroBase():
#     factEuroBase = spark.table("factEuroBase").select("MOP", "GrossWrittenPremium")

#     new_records_df = df.join(
#         factEuroBase, on=["MOP", "GrossWrittenPremium"], how="left_anti"
#     ).select("MOP", "GrossWrittenPremium")

#     dimAccount = spark.table("dimAccount")
#     dimPolicy = spark.table("dimPolicy")
#     dimEntity = spark.table("dimEntity")
#     dimLOB = spark.table("dimLOB")
#     dimYOA = spark.table("dimYOA")
#     dimCurrency = spark.table("dimCurrency")
#     dimAccountingPeriod = spark.table("dimAccountingPeriod")

#     data = []
#     columns = ['fk_AccountID','fk_PolicyID','fk_EntityID','fk_LOBID','fk_YOAID','fk_CurrencyID','fk_PeriodID','MOP','GrossWrittenPremium']

#     for row in new_records_df.collect():
        
#         fk_AccountID = dimAccount.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             (dimAccount.AccountType == when(df.Binder == 1, 'Binder').otherwise('Non Binder'))
#         ).select("pk_AccountID").first()["pk_AccountID"]

       
#         fk_PolicyID = dimPolicy.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             (dimPolicy.PolicyReference == df.PolicyReference) & (dimPolicy.InceptionDate == df.InceptionDate)
#         ).select("pk_PolicyID").first()["pk_PolicyID"]

        
#         fk_EntityID = dimEntity.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             dimEntity.SyndicateNumber == df.SyndicateNumber
#         ).select("pk_EntityID").first()["pk_EntityID"]

        
#         fk_LOBID = dimLOB.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             (dimLOB.TrifocusGroup == df.TriFocusGroup) & (dimLOB.Department == df.Department)
#         ).select("pk_LOBID").first()["pk_LOBID"]

        
#         fk_YOAID = dimYOA.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             dimYOA.PolicyYOA == df.PolicyYOA
#         ).select("pk_YOAID").first()["pk_YOAID"]

       
#         fk_CurrencyID = dimCurrency.join(
#             df.filter((df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])),
#             dimCurrency.PolicySettlementCurrency == df.PolicySettlementCurrency
#         ).select("pk_CurrencyID").first()["pk_CurrencyID"]

        
#         InceptionDate = df.filter(
#             (df.MOP == row['MOP']) & (df.GrossWrittenPremium == row['GrossWrittenPremium'])
#         ).withColumn(
#             "Month", date_format(to_date(substring("InceptionDate", 1, 10), "dd-MM-yyyy"), "M")
#         ).withColumn(
#             "Year", date_format(to_date(substring("InceptionDate", 1, 10), "dd-MM-yyyy"), "y")
#         ).select("Month", "Year").distinct()

#         fk_PeriodID = dimAccountingPeriod.join(
#             InceptionDate,
#             (dimAccountingPeriod.AccountingMonth == InceptionDate.Month) &
#             (dimAccountingPeriod.YearName == InceptionDate.Year)
#         ).select("pk_PeriodID").first()["pk_PeriodID"]

        
#         data.append([
#             fk_AccountID, fk_PolicyID, fk_EntityID, fk_LOBID,
#             fk_YOAID, fk_CurrencyID, fk_PeriodID, row['MOP'], row['GrossWrittenPremium']
#         ])

#     new_fact_df = spark.createDataFrame(data, schema=columns)

    
#     factEuroBase.unionByName(new_fact_df).write.mode('append').saveAsTable("factEuroBase")


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

##### optimizing Pyspark code

In [0]:
# def get_account_id(dimAccount,df, mop, grosswrittenpremium):
    
#     accountid = dimAccount.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             (dimAccount.AccountType == when(df.Binder == 1, 'Binder').otherwise('Non Binder'))
#         ).select("pk_AccountID").first()["pk_AccountID"]
    
#     return accountid

# def get_policy_id(dimPolicy, df, mop, grosswrittenpremium):

#     policyid = dimPolicy.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             (dimPolicy.PolicyReference == df.PolicyReference) & (dimPolicy.InceptionDate == df.InceptionDate)
#         ).select("pk_PolicyID").first()["pk_PolicyID"]
    
#     return policyid

# def get_entity_id(dimEntity, df, mop, grosswrittenpremium):

#     entityid = dimEntity.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             dimEntity.SyndicateNumber == df.SyndicateNumber
#         ).select("pk_EntityID").first()["pk_EntityID"]
    
#     return entityid

# def get_lob_id(dimLOB, df, mop, grosswrittenpremium):

#     lobid = dimLOB.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             (dimLOB.TrifocusGroup == df.TriFocusGroup) & (dimLOB.Department == df.Department)
#         ).select("pk_LOBID").first()["pk_LOBID"]

#     return lobid

# def get_yoa_id(dimYOA, df, mop, grosswrittenpremium):

#     yoaid = dimYOA.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             dimYOA.PolicyYOA == df.PolicyYOA
#         ).select("pk_YOAID").first()["pk_YOAID"]

#     return yoaid

# def get_currency_id(dimCurrency, df, mop, grosswrittenpremium):

#     currencyid = dimCurrency.join(
#             df.filter((df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)),
#             dimCurrency.PolicySettlementCurrency == df.PolicySettlementCurrency
#         ).select("pk_CurrencyID").first()["pk_CurrencyID"]
    
#     return currencyid

# def get_period_id(dimAccountingPeriod, df, mop, grosswrittenpremium):

#     InceptionDate = df.filter(
#             (df.MOP == mop) & (df.GrossWrittenPremium == grosswrittenpremium)
#         ).withColumn(
#             "Month", date_format(to_date(substring("InceptionDate", 1, 10), "dd-MM-yyyy"), "M")
#         ).withColumn(
#             "Year", date_format(to_date(substring("InceptionDate", 1, 10), "dd-MM-yyyy"), "y")
#         ).select("Month", "Year").distinct()

#     periodid = dimAccountingPeriod.join(
#         InceptionDate,
#         (dimAccountingPeriod.AccountingMonth == InceptionDate.Month) &
#         (dimAccountingPeriod.YearName == InceptionDate.Year)
#     ).select("pk_PeriodID").first()["pk_PeriodID"]

#     return periodid

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
# def load_new_records():

#     factEuroBase = spark.table("factEuroBase").select("fk_AccountID","fk_PolicyID","fk_EntityID","fk_LOBID","fk_YOAID","fk_CurrencyID","fk_PeriodID","MOP","GrossWrittenPremium")

#     df = spark.read.format("delta").load("abfss://silver@azureblobstorage404.dfs.core.windows.net/")

#     new_records_df = df.join(
#         factEuroBase, on=["MOP", "GrossWrittenPremium"], how="left_anti"
#     ).select("MOP", "GrossWrittenPremium")

#     factEurobase_n = factEuroBase.unionByName(new_records_df,allowMissingColumns=True)

#     factEurobase_n = factEurobase_n.fillna(0)

#     factEurobase_n.write.mode('append').saveAsTable("factEuroBase")


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
# def LoadFactEuroBase():

#     load_new_records()

#     df = spark.read.format("delta").load("abfss://silver@azureblobstorage404.dfs.core.windows.net/")
#     factEuroBase = spark.table("factEuroBase")
    
#     dimAccount = spark.table("dimAccount")
#     dimPolicy = spark.table("dimPolicy")
#     dimEntity = spark.table("dimEntity")
#     dimLOB = spark.table("dimLOB")
#     dimYOA = spark.table("dimYOA")
#     dimCurrency = spark.table("dimCurrency")
#     dimAccountingPeriod = spark.table("dimAccountingPeriod")

#     for row in factEuroBase.filter((factEuroBase.fk_AccountID == 0) & (factEuroBase.fk_PolicyID == 0) & (factEuroBase.fk_EntityID == 0) & 
#                                    (factEuroBase.fk_LOBID == 0) & (factEuroBase.fk_YOAID == 0) & (factEuroBase.fk_CurrencyID == 0)).collect():


#         fk_AccountID = get_account_id(dimAccount, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_PolicyID = get_policy_id(dimPolicy, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_EntityID = get_entity_id(dimEntity, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_LOBID = get_lob_id(dimLOB, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_YOAID = get_yoa_id(dimYOA, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_CurrencyID = get_currency_id(dimCurrency, df, row["MOP"],row["GrossWrittenPremium"])
#         fk_PeriodID = get_period_id(dimAccountingPeriod, df, row["MOP"],row["GrossWrittenPremium"])
        

#         factEuroBase.withColumns({
#         "fk_AccountID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_AccountID)
#         ).otherwise(lit(0)),
#         "fk_PolicyID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_PolicyID)
#         ).otherwise(lit(0)),
#         "fk_EntityID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_EntityID)
#         ).otherwise(lit(0)),
#         "fk_LOBID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_LOBID)
#         ).otherwise(lit(0)),
#         "fk_YOAID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_YOAID)
#         ).otherwise(lit(0)),
#         "fk_CurrencyID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_CurrencyID)
#         ).otherwise(lit(0)),
#         "fk_PeriodID": when(
#             (factEuroBase.MOP == row["MOP"]) & (factEuroBase.GrossWrittenPremium == row["GrossWrittenPremium"]),
#             lit(fk_PeriodID)
#         ).otherwise(lit(0))
#         })

#     # factEuroBase.write.mode('append').saveAsTable("factEuroBase")


org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

In [0]:
# LoadFactEuroBase()

org.apache.spark.sql.catalyst.parser.ParseException: 
[DATATYPE_MISSING_SIZE] DataType "VARCHAR" requires a length parameter, for example "VARCHAR"(10). Please specify the length. SQLSTATE: 42K01
== SQL (line 3, position 17) ==
    update_date VARCHAR
                ^^^^^^^

	at org.apache.spark.sql.errors.QueryParsingErrors$.charTypeMissingLengthError(QueryParsingErrors.scala:307)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:287)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:154)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:144)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:185)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:249)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTyp

##### optimizing the pyspark code using (JOINS ONLY)

In [0]:
def loadfacteurobase():

    df = spark.read.format("delta")\
            .load(ds_path)

    facteurobase = spark.table("facteurobase").select("MOP", "GrossWrittenPremium", "fk_PolicyID","fk_AccountID","fk_EntityID","fk_LOBID","fk_YOAID","fk_CurrencyID","fk_PeriodID")

    dimPolicy = spark.table("dimPolicy")
    dimAccount = spark.table("dimAccount")
    dimCurrency = spark.table("dimCurrency")
    dimPeriod = spark.table("dimAccountingPeriod")
    dimEntity = spark.table("dimEntity")
    dimLOB = spark.table("dimLOB")
    dimYOA = spark.table("dimYOA")
    watermark = spark.table("watermark")

    if watermark.rdd.isEmpty():
        watermark = get_default_watermark_date()
    
    watermark_max_date = watermark.agg(max("update_date").alias("max_update_date")).collect()[0]["max_update_date"]


    df_result = df.join(facteurobase.select("MOP", "GrossWrittenPremium"), on = [df.MOP == facteurobase.MOP, df.GrossWrittenPremium == facteurobase.GrossWrittenPremium], how = "left_anti")\
    .join(dimPolicy.select("pk_PolicyID","PolicyReference","InceptionDate"), on = [df.PolicyReference == dimPolicy.PolicyReference , df.InceptionDate == dimPolicy.InceptionDate], how = "left")\
        .join(dimAccount.select("pk_AccountID","AccountType"), on = (df.Binder == when(dimAccount.AccountType == "Binder", 1).otherwise(0)), how = "left")\
            .join(dimCurrency.select("pk_CurrencyID","PolicySettlementCurrency"), on = (df.PolicySettlementCurrency == dimCurrency.PolicySettlementCurrency), how = "left")\
                .join(dimEntity.select("pk_EntityID","SyndicateNumber"), on = (df.SyndicateNumber == dimEntity.SyndicateNumber), how = "left")\
                        .join(dimLOB.select("pk_LOBID","TrifocusGroup","Department"), on = [df.TriFocusGroup == dimLOB.TrifocusGroup, df.Department == dimLOB.Department], how = "left")\
                            .join(dimYOA.select("pk_YOAID","PolicyYOA"), on = (df.PolicyYOA == dimYOA.PolicyYOA), how = "left")\
                                .join(dimPeriod.select("pk_PeriodID","AccountingMonth","YearName"), on = [substring(df.InceptionDate,4,2) == dimPeriod.AccountingMonth , substring(df.InceptionDate,7,4) == dimPeriod.YearName], how = "left")\
                                    .where(to_date(df.report_date, "dd-MM-yyyy") >= to_date(lit(watermark_max_date), "dd-MM-yyyy"))


    res = df_result.select(
    col("MOP"),
    col("GrossWrittenPremium"),
    col("pk_PolicyID").cast("int").alias("fk_PolicyID"),
    col("pk_AccountID").cast("int").alias("fk_AccountID"),
    col("pk_EntityID").cast("int").alias("fk_EntityID"),
    col("pk_LOBID").cast("int").alias("fk_LOBID"),
    col("pk_YOAID").cast("int").alias("fk_YOAID"),
    col("pk_CurrencyID").cast("int").alias("fk_CurrencyID"),
    col("pk_PeriodID").cast("int").alias("fk_PeriodID"),  
    )

    facteurobase_new = facteurobase.unionByName(res)
    facteurobase_new.write.mode("overwrite").saveAsTable("facteurobase")
    update_watermark_date()

In [0]:
loadfacteurobase()

Watermark DataFrame is empty. Initializing with default date (01-01-1900).
New max date found: 12-09-2018. Updating watermark table.
